In [2]:
import numpy as np
from tqdm.auto import tqdm
import os
import glob
import shutil

import librosa
import torchaudio as ta
import torch
import gc

In [5]:
clips_path = "./dataset/cv-corpus-5.1-2020-06-22/en/clips/"
file_nums = []
for file in tqdm(os.listdir(clips_path)):
    num = file.split("_")[3]
    num = int(num.split(".")[0])
    file_nums.append(num)
file_nums = np.asarray(file_nums)

In [6]:
type(file_nums)

numpy.ndarray

In [8]:
np.save("./dataset_loader_files/signal_paths_nums_save.npy", file_nums)

In [11]:
noise_dir = "./dataset/UrbanSound8K/audio/"
file_num = 0
for root, dirs, files in os.walk(noise_dir):
    for name in files:
        if name.endswith(".wav"):
            os.rename(os.path.join(root, name), os.path.join(root, str(file_num)+".wav"))
        file_num += 1

In [13]:
copy_dir = "./dataset/UrbanSound8K/all_files/"
for root, dirs, files in os.walk(noise_dir):
    for name in files:
        if name.endswith(".wav"):
            shutil.move(os.path.join(root, name), os.path.join(copy_dir, name))

In [4]:
noise_paths = []
noise_dir = "./dataset/UrbanSound8K-Resampled//audio/"
for root, dirs, files in os.walk(noise_dir):
    for name in files:
        if name.endswith(".wav"):
            noise_paths.append(os.path.join(root, name))
noise_paths_arr = np.asarray(noise_paths)

In [6]:
np.save("dataset_loader_files/noise_paths_resampled_save.npy", noise_paths_arr)

In [15]:
file_nums = []
for file in tqdm(os.listdir(copy_dir)):
    num = int(file.split(".")[0])
    file_nums.append(num)
file_nums = np.asarray(file_nums)

In [16]:
len(file_nums)

8732

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("./dataset/UrbanSound8K/metadata/UrbanSound8K.csv")

In [20]:
df = df[["slice_file_name", "fold"]]

In [26]:
base_dir = "./dataset/UrbanSound8K/audio/"
current_dir = "./dataset/UrbanSound8K/all_files/"
for index, row in tqdm(df.iterrows()):
    file_name = row["slice_file_name"]
    fold = row["fold"]
    new_path = os.path.join(base_dir, f"fold{fold}", file_name)
    old_path = os.path.join(current_dir, file_name)
    shutil.copy2(old_path, new_path)
    

FileNotFoundError: [Errno 2] No such file or directory: './dataset/UrbanSound8K/all_files/100032-3-0-0.wav'

In [2]:
ta.set_audio_backend("sox_io")

In [12]:
clips_path = "./dataset/cv-corpus-5.1-2020-06-22-Resampled/en/clips/"
file_nums = []

batch_len = 12
default_sr = 16000
resampler =  ta.transforms.Resample
lst_dir_tqdm = tqdm(enumerate(os.listdir(clips_path)), total=1427394)
for index, file in lst_dir_tqdm:
    
    aud_data, sr = ta.load(os.path.join(clips_path, file))
    if not sr==16000:
        aud_data = aud_data.to("cpu")
        resampler =  ta.transforms.Resample(orig_freq=sr, new_freq=default_sr)
        aud_data = resampler(aud_data)
        ta.save(os.path.join(clips_path, file), aud_data.to("cpu"), sample_rate=default_sr)
    if index/1000 == 0:
        gc.collect()

In [51]:
clips_path = "./dataset/UrbanSound8K-Resampled/audio/*"

batch_len = 12
default_sr = 16000
resampler =  ta.transforms.Resample
lst_dir_tqdm = tqdm(enumerate(glob.glob(clips_path)))
for index, folder in lst_dir_tqdm:
    removed = 0
    aud_files = glob.glob(os.path.join(folder, "*.wav"))
    aud_files_tqdm = tqdm(aud_files)
    for file in aud_files_tqdm:
        try:
            aud_data, sr = ta.load_wav(file)
        except RuntimeError:
            os.remove(file)
            removed += 1
            aud_files_tqdm.set_postfix(removed = removed)
        if not sr==16000:
            aud_data = aud_data.to("cpu")
            resampler =  ta.transforms.Resample(orig_freq=sr, new_freq=default_sr)
            aud_data = resampler(aud_data)
            ta.save(file, aud_data.to("cpu"), sample_rate=default_sr)
#         if index/1000 == 0:
#             gc.collect()

In [32]:
file

'./dataset/UrbanSound8K-Resampled/audio/fold5/172460-9-0-24.wav'

In [19]:
lst = glob.glob("./dataset/UrbanSound8K-Resampled/audio/*")

In [20]:
lst

['./dataset/UrbanSound8K-Resampled/audio/fold5',
 './dataset/UrbanSound8K-Resampled/audio/fold1',
 './dataset/UrbanSound8K-Resampled/audio/fold10',
 './dataset/UrbanSound8K-Resampled/audio/fold4',
 './dataset/UrbanSound8K-Resampled/audio/fold7',
 './dataset/UrbanSound8K-Resampled/audio/fold2',
 './dataset/UrbanSound8K-Resampled/audio/fold3',
 './dataset/UrbanSound8K-Resampled/audio/fold6',
 './dataset/UrbanSound8K-Resampled/audio/fold8',
 './dataset/UrbanSound8K-Resampled/audio/fold9',
 './dataset/UrbanSound8K-Resampled/audio/UrbanSound8K.csv']

In [35]:
@torch.jit.trace
def resampled_signal_writer(aud_data, sr, default_sr):

    aud_data = aud_data.to("cuda")
    #     print(aud_data.shape)
    resampler =  ta.transforms.Resample(orig_freq=torch.tensor(sr), new_freq=torch.tensor(default_sr))
    aud_data = resampler(aud_data)
    return aud_data
    
        


TypeError: trace() missing 1 required positional argument: 'example_inputs'

In [ ]:
clips_path = "./dataset/cv-corpus-5.1-2020-06-22-Resampled/en/clips/"
lst_dir_tqdm = tqdm(enumerate(os.listdir(clips_path)), total=1427394)

for index, file in lst_dir_tqdm:
    read_path = os.path.join(clips_path, file)
    aud_data, sr = ta.load(path)
    aud_data = resampled_signal_writer(aud_data, sr, default_sr)
    ta.save(path, aud_data.to("cpu"), sample_rate=default_sr)
    

In [ ]:
clips_path = "./dataset/cv-corpus-5.1-2020-06-22-Resampled/en/clips/"
file_nums = []

workers = 8
default_sr = 16000
multi_pool = torch.multiprocessing.Pool(processes=workers)

def resampler(aud_data_sr):
    global default_sr
    aud_data, sr = aud_data_sr
    resampler =  ta.transforms.Resample(orig_freq=sr, new_freq=default_sr)
    aud_data = resampler(aud_data)
    

lst_dir_tqdm = tqdm(enumerate(os.listdir(clips_path)), total=1427394)
for index, file in lst_dir_tqdm:
    aud_stack = []
    for i in range(workers):
        
        aud_data, sr = ta.load(os.path.join(clips_path, file))
    #     print(aud_data.shape)
        resampler =  ta.transforms.Resample(orig_freq=sr, new_freq=default_sr)
        aud_data = resampler(aud_data)
        ta.save(os.path.join(clips_path, file), aud_data, sample_rate=default_sr)

In [10]:
len(os.listdir(clips_path))

1427394

In [22]:
aud_data.shape

torch.Size([76032])